In [1]:
from itertools import permutations
from constants import *

import ollama
import os

In [2]:
RESULTS_FOLDER = "results"
GENERATED_CODE_FOLDER = os.path.join(RESULTS_FOLDER, "generated_code")
ACCEPTED_CODE_FOLDER = os.path.join(RESULTS_FOLDER, "accepted_code")
COMPILE_FAIL_FOLDER = os.path.join(RESULTS_FOLDER, "compile_fail")
SYNTAX_FAIL_FOLDER = os.path.join(RESULTS_FOLDER, "syntax_fail")
FOLLOWUP_RESP_FOLDER = os.path.join(RESULTS_FOLDER, "generated_followup")
MODEL = "deepseekcoder7b"
LANG2LANG = list(permutations(LANGS.keys(), 2))

In [3]:
os.makedirs(FOLLOWUP_RESP_FOLDER, exist_ok=True)

In [4]:
def get_prompt(system_prompt, user_prompt1, assistant_prompt, user_prompt2):
    prompt = []
    if system_prompt:
        prompt.append({"role": "system", "content": system_prompt})
    prompt.append({"role": "user", "content": user_prompt1})
    prompt.append({"role": "user", "content": assistant_prompt})
    prompt.append({"role": "user", "content": user_prompt2})
    return prompt

In [5]:
prompt = "Convert the following code from {} to {}. This is the requirement for the code - {}\n```{}```\n"

In [6]:
compile_prompt = """When compiling the code enclosed in backticks, the following error occurred -

```{}```

Explain the error in a single line.
Then explain the fix in single line.
Then rewrite the corrected code in a single code block.
Ensure you write the entire code and your response should only contain one fenced code block."""

In [7]:
syntax_prompt = """When checking the syntax of code enclosed in backticks, the following issue was highlighted -

```{}```

Explain the error in a single line.
Then explain the fix in single line.
Then rewrite the corrected code in a single code block.
Ensure you write the entire code and your response should only contain one fenced code block."""

In [ ]:
ollama.pull(OLLAMA_TAGS[MODEL])

In [ ]:
for TASK in TASKS.keys():
    for idx, (lang1, lang2) in enumerate(LANG2LANG):
        filename = f"{MODEL}-{TASK}-{lang1.lower()}-{lang2.lower()}"
        compile_fail = False
        syntax_fail = False
        if os.path.exists(os.path.join(COMPILE_FAIL_FOLDER, f"{filename}.txt")):
            compile_fail = True
        if os.path.exists(os.path.join(SYNTAX_FAIL_FOLDER, f"{filename}.txt")):
            syntax_fail = True
        if os.path.exists(os.path.join(FOLLOWUP_RESP_FOLDER, f"{filename}.md")):
            continue
        if not compile_fail and not syntax_fail:
            continue
        formatted_prompt = prompt.format(
            LANGS[lang1],
            LANGS[lang2],
            TASK_DESCRIPTION[TASK],
            REFERENCE_CODE[lang1][TASK],
        )
        llm_response = (
            open(os.path.join(GENERATED_CODE_FOLDER, f"{filename}.md"), "r")
            .read()
            .split("```")
        )
        formatted_code = open(
            os.path.join(ACCEPTED_CODE_FOLDER, f"{filename}.{lang2.lower()}"), "r"
        ).read()
        formatted_response = f"{llm_response[0]}```{lang2.lower()}\n{formatted_code}\n```{llm_response[2]}"
        if compile_fail:
            compile_error = open(
                os.path.join(COMPILE_FAIL_FOLDER, f"{filename}.txt"), "r"
            ).read()
            formatted_followup = compile_prompt.format(compile_error)
        elif syntax_fail:
            syntax_error = open(
                os.path.join(SYNTAX_FAIL_FOLDER, f"{filename}.txt"), "r"
            ).read()
            formatted_followup = syntax_prompt.format(syntax_error)
        if OLLAMA_SYSTEM_PROMPT_SUPPORT[MODEL]:
            chat_prompt = get_prompt(
                "You are a helpful code conversion assistant.",
                formatted_prompt,
                formatted_response,
                formatted_followup,
            )
        else:
            chat_prompt = get_prompt(
                None, formatted_prompt, formatted_response, formatted_followup
            )
        try:
            rsp = ollama.chat(
                OLLAMA_TAGS[MODEL],
                messages=chat_prompt,
                options={"seed": 42, "timeout": 60},
            )
            with open(os.path.join(FOLLOWUP_RESP_FOLDER, f"{filename}.md"), "w") as fp:
                fp.write(rsp["message"]["content"] + "\n")
            print(f"completed {MODEL} {TASK} {lang1} {lang2}")
        except:
            try:
                rsp = ollama.chat(
                    OLLAMA_TAGS[MODEL],
                    messages=chat_prompt,
                    options={"seed": 41, "timeout": 60},
                )
                with open(
                    os.path.join(FOLLOWUP_RESP_FOLDER, f"{filename}.md"), "w"
                ) as fp:
                    fp.write(rsp["message"]["content"] + "\n")
                print(f"completed {MODEL} {TASK} {lang1} {lang2}")
            except:
                with open(
                    os.path.join(FOLLOWUP_RESP_FOLDER, f"{filename}.md"), "w"
                ) as fp:
                    fp.write("timeout\n")
                print(f"timeout {MODEL} {TASK} {lang1} {lang2}")